**⭐ 1. What This Pattern Solves**

This pattern finds answers in contiguous subarrays/substrings while scanning the data ONCE.

Used for:

Longest/shortest substring with conditions

Maximum/minimum sum of subarray

Finding number of valid subarrays

Counting elements within a window

Time-series rolling window operations

Stream processing (log analysis)

Real-time metrics (hits per minute, API rate limits)

Event sessionization

Dynamic filtering

Sliding window transforms O(n²) brute force into O(n).

**⭐ 2. SQL Equivalent**

In [0]:
%sql

-- 1. Window frames
SELECT 
  user,
  COUNT(*) OVER (
    PARTITION BY user 
    ORDER BY ts 
    RANGE BETWEEN INTERVAL '5' MINUTE PRECEDING AND CURRENT ROW
  )
FROM logs;

-- 2. Between conditions
WHERE ts BETWEEN start AND end

**⭐ 3. Core Idea**

Maintain a window defined by two pointers:

left = 0
right = 0


Expand right to grow the window.
Shrink left when window becomes invalid.
Record results while maintaining some state (sum, count, freq-map, etc.).

**⭐ 4. Template Code (MEMORIZE THIS)**

In [0]:
## Template A — Variable-sized sliding window (most common)

left = 0

for right in range(len(arr)):

    # include arr[right] into window
    add(arr[right])

    # shrink window until valid
    while window_is_invalid():
        remove(arr[left])
        left += 1

    # now window [left:right] is valid
    update_answer()

In [0]:
## Template B — Fixed-size window (simple)

left = 0
window_sum = 0

for right in range(len(arr)):
    window_sum += arr[right]

    if right - left + 1 > k:
        window_sum -= arr[left]
        left += 1

    if right - left + 1 == k:
        update_answer(window_sum)

In [0]:
## Template C — Using frequency map

from collections import defaultdict

count = defaultdict(int)
left = 0

for right in range(len(s)):
    count[s[right]] += 1

    while window_invalid(count):
        count[s[left]] -= 1
        left += 1

    update_answer(left, right)

## This is the backbone for substring problems.
## You must be able to write Template A instantly.

**⭐ 5. Detailed Example**

In [0]:
## Problem : Find the length of the longest substring with no repeating characters.

#Input:
s = "abcabcbb"

## Apply template:

from collections import defaultdict

freq = defaultdict(int)
left = 0
best = 0

for right in range(len(s)):
    freq[s[right]] += 1

    # shrink until valid
    while freq[s[right]] > 1:
        freq[s[left]] -= 1
        left += 1

    best = max(best, right - left + 1)

**⭐ 6. Mini Practice Problems (Do without looking)**

Problem 1 — Fixed window max sum

Input: [2,1,5,1,3,2], k=3
Expected: max sum = 9

Use Template B.

Problem 2 — Longest substring with at most K distinct characters

Input: "eceba", k=2
Expected = 3 ("ece")

Use Template A + freq map.

Problem 3 — Count subarrays with sum ≤ target

Input: [1,3,2,5,1], target = 7

Use growing window until sum > target, then shrink.

Problem 4 — Longest 1s after flipping at most k zeros

Input: [1,1,1,0,0,1,1], k=1

Classic interview problem.

**⭐ 7. Full Data Engineering Problem**

In [0]:
## Problem : You receive ordered API hits:

ts = [1, 2, 3, 10, 11, 12, 13]
window = 3  # seconds

## Task: Compute number of hits in each rolling 3-second window.

left = 0

for right in range(len(ts)):
    # expand window by including ts[right]

    while ts[right] - ts[left] > window:
        left += 1

    # window is valid; ts[left:right] inclusive
    count = right - left + 1
    print(ts[right], count)


**This is used in:**

rate limiting

spike detection

real-time metrics

event window aggregations

**⭐ 8. Time & Space Complexity**

Time: O(n) — each pointer moves once

Space: O(1) or O(k) (if using freq map)

**⭐ 9. Common Pitfalls & Mistakes**

❌ Not shrinking the window → infinite growth
❌ Shrinking incorrectly → skipping valid windows
❌ Using nested loops inside → defeats purpose
❌ Forgetting to subtract removed elements
❌ Mixing up fixed vs variable window patterns

✔ Always confirm if the window is:

fixed size? → Template B

variable size with constraint? → Template A